In [1]:
import numpy as np
from conv import Conv3x3
from maxpool import MaxPool2
from softmax import Softmax
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

In [2]:
train_losses = []
test_losses = []

Define the path to the dataset

In [3]:
data_path = "Fdata1"
print (data_path)

Fdata1


In [4]:
num_images = 0
class_names = []
img_size = (128, 128)

Initialize empty lists for images and labels

In [5]:
images = []
labels = []

In [6]:
for class_name in os.listdir(data_path):
    class_dir = os.path.join(data_path, class_name)
    if os.path.isdir(class_dir):
        class_images = os.listdir(class_dir)
        num_images += len(class_images)
        class_names.append(class_name)
        
        # Iterate over image files in class directory
        for image_name in class_images:
            image_path = os.path.join(class_dir, image_name)
            
            # Load image and append to images list
            image = Image.open(image_path)
            image = image.resize(img_size)
            
            # Convert to grayscale
            gray_image = image.convert('L')
            image_data = np.array(gray_image)
            images.append(image_data)
            
            # Append label to labels list
            labels.append(class_names.index(class_name))

In [7]:
num_classes = len(class_names)
images = np.array(images)
labels = np.array(labels)

In [8]:
print(images[2].shape)

(128, 128)


In [9]:
print(f"Found {num_images} images belonging to {num_classes} classes: {class_names}")
# Load the data and split it into training and validation sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, stratify=labels)

Found 80 images belonging to 3 classes: ['Akshay Kumar', 'Amitabh Bachchan', 'Urvish']


In [10]:
conv = Conv3x3(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool2()                  # 26x26x8 -> 13x13x8
softmax = Softmax(63 * 63 * 8, num_classes ) # 13x13x8 -> 10

In [11]:
def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

In [13]:
  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

NameError: name 'out' is not defined

In [ ]:
  return out, loss, acc

In [14]:
def train(im, label, lr=.005):
  '''
  Completes a full training step on the given image and label.
  Returns the cross-entropy loss and accuracy.
  - image is a 2d numpy array
  - label is a digit
  - lr is the learning rate
  '''
  # Forward
  out, loss, acc = forward(im, label)

In [15]:
  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

NameError: name 'out' is not defined

In [ ]:
  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

In [ ]:
  return loss, acc

In [ ]:
print('MNIST CNN initialized!')

Train the CNN for 3 epochs

In [ ]:
for epoch in range(3):
  print('--- Epoch %d ---' % (epoch + 1))

In [ ]:
  # Shuffle the training data
  permutation = np.random.permutation(len(train_images))
  train_images = train_images[permutation]
  train_labels = train_labels[permutation]

In [ ]:
  # Train!
  loss = 0
  num_correct = 0
  for i, (im, label) in enumerate(zip(train_images, train_labels)):
    if i % 100 == 99:
      print(
        '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
        (i + 1, loss / 100, num_correct)
      )
      loss = 0
      num_correct = 0
    l, acc = train(im, label)
    loss += l
    num_correct += acc

In [ ]:
    # Calculate and store loss for the current epoch
    train_loss = loss / len(train_images)
    train_losses.append(train_loss)

    # Calculate and store test loss for the current epoch
    test_loss = 0
    for im, label in zip(test_images, test_labels):
        _, l, _ = forward(im, label)
        test_loss += l
    test_loss /= len(test_images)
    test_losses.append(test_loss)

Test the CNN

In [ ]:
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

In [ ]:
num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

Plot the loss vs epoch graph

In [ ]:
epochs = np.arange(1, len(train_losses)+1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, test_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def predict_image(image_path):
  # Load the image you want to predict
  
  image = Image.open(image_path)
  plt.imshow(image)
  plt.show()
  image = image.resize((128, 128))
  gray_image = image.convert('L')
  image_data = np.array(gray_image)

In [ ]:
  # Forward pass through the CNN to get prediction
  out = conv.forward((image_data / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)
  prediction = np.argmax(out)

In [ ]:
  # Print the predicted class
  # class_names = ["class1", "class2", "class3", ...] # replace with your own class names
  print("Predicted class:", class_names[prediction])

In [ ]:
predict_image("FData1\Alia Bhatt\Alia Bhatt_4.jpg")
predict_image("FData1\Alia Bhatt\Alia Bhatt_7.jpg")
predict_image("FData1\Akshay Kumar\Akshay Kumar_4.jpg")
predict_image("FData1\Akshay Kumar\Akshay Kumar_4.jpg")
predict_image("FData1\Alexandra Daddario\Alexandra Daddario_4.jpg")
predict_image("FData1\Amitabh Bachchan\Amitabh Bachchan_4.jpg")